# DATASCI W261: Machine Learning at Scale
## Assignment Week 4
Miki Seltzer (miki.seltzer@berkeley.edu)<br>
W261-2, Spring 2016<br>
Submission: 

## HW 4.0

### What is MrJob? How is it different to Hadoop MapReduce? 
MRJob is a framework for running MapReduce jobs. It is flexible and allows you to run jobs locally, using Hadoop, or even on AWS. It allows for chaining MapReduce jobs together, while Hadoop MapReduce only allows running one job at a time. Similarly to the Hadoop Streaming API, we can write MapReduce jobs in Python, and utilize all of the libraries available to Python.

### What are the mapper_init(), mapper_final(), combiner_final(), reducer_final() methods? When are they called?
- mapper_init(): This runs before the mapper processes any input. It can be used to initialize mapper-specific helper structures.
- mapper_final(): This runs after the mapper gets to the end of the input stream. This can be used to store variables that should only be emitted after all data has been processed, such as a total.
- combiner_final(): This is similar to mapper_final(), but runs at the end of the input stream in the combiner.
- reducer_final(): Also similar to mapper_final(), but runs at the end of the input stream in the reducer.

## HW 4.1

### What is serialization in the context of MrJob or Hadoop? 
Serialization is the process of translating an object into a series of bytes for sending it where it's needed.

### When it used in these frameworks? 
Each job has specified protocols that serialize and deserialize bytes as they are sent between different steps of the MapReduce jobs.

### What is the default serialization mode for input and outputs for MrJob? 
The default protocol for serialization is RawValueProtocol, which just reads each record as a string.

## HW4.2: Write the Python code to preprocess the Microsoft logfile data on a single node into the appropriate format.

In [17]:
import csv

# Create files to write the processed data to
outfile = open('anonymous-msweb-preprocess.data', 'wb')
outWriter = csv.writer(outfile)
attributes = open('attributes.csv', 'wb')
attWriter = csv.writer(attributes)

with open('anonymous-msweb.data', 'r') as infile:
    for line in csv.reader(infile):
        
        # Check if this line is an attribute
        # If it is, write to the attributes file
        if line[0] == 'A':
            attWriter.writerow([line[1], line[3], line[4]])
            
        # Check if this line is a visitor ID
        # If it is, save it to memory
        elif line[0] == 'C':
            visitor_id = line[2]
        
        # Check if this line is a visit with a page ID
        # If it is, write it with the current visitor_id
        elif line[0] == 'V':
            outWriter.writerow([line[0], line[1], line[2], 'C', visitor_id])

outfile.close()
attributes.close()

In [18]:
# Test out our code above